# Imports


In [1]:
import numpy as np
import torch
from torchvision import transforms
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import random
import json
import os

# Data Augmentation

In [2]:
'''
    Augment the image by flipping, ColorJitter, GaussianBlur, and change the bounding box accordingly
'''

def augment_image(image, bbox):
    pil_image = Image.fromarray(image)
    draw = ImageDraw.Draw(pil_image)
    
    # Convert bbox to format required for PIL (left, upper, right, lower)
    #(x_min, y_min, x_max, y_max) = bbox
    bbox_pil = [bbox[0], bbox[1], bbox[2], bbox[3]]

    
    # Define the augmentation pipeline
    flag = random.randint(0, 3)
    if flag == 0:
        image = np.flip(image, [0])
        bbox_pil[1] = pil_image.size[1] - bbox_pil[1]
        bbox_pil[3] = pil_image.size[1] - bbox_pil[3]
    elif flag == 1:
        image = np.flip(image, [1])
        bbox_pil[0] = pil_image.size[0] - bbox_pil[0]
        bbox_pil[2] = pil_image.size[0] - bbox_pil[2]
    elif flag == 2:
        image = np.flip(image, [0, 1])
        bbox_pil[0] = pil_image.size[0] - bbox_pil[0]
        bbox_pil[2] = pil_image.size[0] - bbox_pil[2]
        bbox_pil[1] = pil_image.size[1] - bbox_pil[1]
        bbox_pil[3] = pil_image.size[1] - bbox_pil[3]
    else:
        pass # same image
    

    color_jitter = transforms.ColorJitter(
        brightness=0.2,    # Randomly change brightness
        contrast=0.2,      # Randomly change contrast
        saturation=0.2,    # Randomly change saturation
        hue=0.1            # Randomly change hue
    )

    image = color_jitter(Image.fromarray(image))
    image = np.array(image)

    blur = transforms.GaussianBlur(kernel_size=(5,25), sigma=(0.1, 5))
    image = blur(Image.fromarray(image))
    image = np.array(image)

    
    return image, bbox_pil

In [3]:
'''
    Convert the bounding box from pixel format to normalized format
'''
def normalized_to_pixel_bbox(data, img_width, img_height):
    x_min = (data["x"] / 100) * img_width
    y_min = (data["y"] / 100) * img_height
    width = (data["width"] / 100) * img_width
    height = (data["height"] / 100) * img_height
    x_max = x_min + width
    y_max = y_min + height
    # print(x_min, y_min, x_max, y_max)
    return [x_min, y_min, x_max, y_max]

In [4]:
'''
    Adjust the bounding box to the new image size
'''
def adjust_bounding_boxes(bboxes, original_size, target_size):
    original_width, original_height = original_size
    target_width, target_height = target_size
    
    scale_width = target_width / original_width
    scale_height = target_height / original_height
    
    adjusted_bboxes = []
    for bbox in bboxes:
        x_min = bbox['x_min']
        y_min = bbox['y_min']
        x_max = bbox['x_max']
        y_max = bbox['y_max']
        x_min_adj = x_min * scale_width
        y_min_adj = y_min * scale_height
        x_max_adj = x_max * scale_width
        y_max_adj = y_max * scale_height
        
        adjusted_bboxes.append((x_min_adj, y_min_adj, x_max_adj, y_max_adj))
    
    return adjusted_bboxes  


# Generate Resized Training Images


In [5]:
def generate_image(num_augmentations, start_index, csv_data, image_path, target_size, has_symbol=True, resize_image=False, validation=False):
      # Update with the actual path
    image = cv2.imread(image_path) # The image is loaded in BGR format
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for visualization

    bbox = normalized_to_pixel_bbox(csv_data, csv_data["original_width"], csv_data["original_height"])

    # Number of augmentations to generate
    augmented_results = []

    for i in range(num_augmentations):
        augmented_image, adjusted_bbox = augment_image(image.copy(), bbox.copy())
        augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2RGB)

        # Resize the augmented image and bounding box
        if resize_image:
            augmented_image = cv2.resize(augmented_image, target_size)
            temp = {
                'x_min': adjusted_bbox[0],
                'y_min': adjusted_bbox[1],
                'x_max': adjusted_bbox[2],
                'y_max': adjusted_bbox[3]
            }
            adjusted_bbox = adjust_bounding_boxes([temp], (csv_data["original_width"], csv_data["original_height"]), target_size)[0]
        # ----------------------------

        augmented_image_np = np.array(augmented_image)
        augmented_results.append((augmented_image_np, adjusted_bbox))

    bbox_annotations = []
    for i, (augmented_image, adjusted_bbox) in enumerate(augmented_results):
        # Save augmented image
        if not os.path.exists('data'):
            os.makedirs('data')
        augmented_image_path = f'data/image_{i+start_index}.jpg'
        if validation:
            if not os.path.exists('validation_data'):
                os.makedirs('validation_data')
            augmented_image_path = f'validation_data/image_{i+start_index}.jpg'
        if resize_image:
            if not os.path.exists('resized_data'):
                os.makedirs('resized_data')
            augmented_image_path = f'resized_data/image_{i+start_index}.jpg'
            if validation:
                if not os.path.exists('resized_validation_data'):
                    os.makedirs('resized_validation_data')
                augmented_image_path = f'resized_validation_data/image_{i+start_index}.jpg'

        
        image_name = f'image_{i+start_index}.jpg'
        cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))


        if has_symbol:
            bbox = {
                'x_min': adjusted_bbox[0],
                'y_min': adjusted_bbox[1],
                'x_max': adjusted_bbox[2],
                'y_max': adjusted_bbox[3]
            }
        else:
            bbox = {
                'x_min': 0,
                'y_min': 0,
                'x_max': 0,
                'y_max': 0
            }

        # Save adjusted bounding box coordinates to the annotations list
        bbox_data = {
            'image_path': image_name,
            'has_symbol': 1 if has_symbol else 0,
            'bbox': bbox
        }
        bbox_annotations.append(bbox_data)
    return bbox_annotations

    

In [6]:
back_data = {
    "x": 5.075424261470773,
    "y": 10.69767441860465,
    "width": 19.721648558857858,
    "height": 31.395348837209305,
    "rotation": 0,
    "rectanglelabels": ["CCP"],
    "original_width": 1036,
    "original_height": 646 
    }
back_1_data = {
    "x":5.0754242614707605,
    "y":9.302325581395342,
    "width":19.14160007183262,
    "height":32.32558139534885,
    "rotation": 0,
    "rectanglelabels": ["CCP"],
    "original_width": 1036,
    "original_height": 646
}

In [7]:
def generate_training(num_augmentations=100):
    image_path = 'original_data/back.bmp'
    target_size = (256, 256)
    bbox_annotations = generate_image(num_augmentations, 0, back_data, image_path, target_size, has_symbol=True, resize_image=True, validation=False)
    image_path = 'original_data/back_1.bmp'
    bbox_annotations += generate_image(num_augmentations, num_augmentations, back_1_data, image_path, target_size, has_symbol=True, resize_image=True, validation=False)
    image_path = 'original_data/front.bmp'
    bbox_annotations += generate_image(num_augmentations, 2 * num_augmentations, back_data, image_path, target_size, has_symbol=False, resize_image=True, validation=False)
    image_path = 'original_data/stain_back.bmp'
    bbox_annotations += generate_image(num_augmentations, 3 * num_augmentations, back_data, image_path, target_size, has_symbol=False, resize_image=True, validation=False)

    print(len(bbox_annotations))
    train_annotations_path = 'resized_data/train_annotations.json'

    with open(train_annotations_path, 'w') as f:
        json.dump(bbox_annotations, f, indent=4)

    print(f"Training images and bounding boxes saved to {train_annotations_path}")

In [ ]:
generate_training()


# Visualize Images

In [8]:
def visualize(image_dir, annotations_path):
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)
    
    for annotation in annotations:
        if annotation['has_symbol'] == 0:
            image_path = f'{image_dir}/{annotation["image_path"]}'
            image = cv2.imread(image_path)
            cv2.imshow('Image', image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            image_path = f'{image_dir}/{annotation["image_path"]}'
            image = cv2.imread(image_path)
            x_min = int(annotation['bbox']['x_min'])
            y_min = int(annotation['bbox']['y_min'])
            x_max = int(annotation['bbox']['x_max'])
            y_max = int(annotation['bbox']['y_max'])
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
            cv2.imshow('Image', image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

# Generate Validation Images

In [ ]:
# image_dir = 'resized_data'
# annotations_path = 'resized_data/annotations.json'
# visualize(image_dir, annotations_path)
# new_dir = 'data'
# new_annotations_path = 'data/annotations.json'
# visualize(new_dir, new_annotations_path)

In [9]:
def generate_validation(num_augmentations=20):    
    image_path = 'original_data/back.bmp'
    target_size = (256, 256)
    val_annotation = generate_image(num_augmentations, 0, back_data, image_path, target_size, has_symbol=True, resize_image=True, validation=True)
    image_path = 'original_data/back_1.bmp'
    val_annotation += generate_image(num_augmentations, num_augmentations, back_1_data, image_path, target_size, has_symbol=True, resize_image=True, validation=True)
    image_path = 'original_data/front.bmp'
    val_annotation += generate_image(num_augmentations, 2 * num_augmentations, back_data, image_path, target_size, has_symbol=False, resize_image=True, validation=True)
    image_path = 'original_data/stain_back.bmp'
    val_annotation += generate_image(num_augmentations, 3 * num_augmentations, back_data, image_path, target_size, has_symbol=False, resize_image=True, validation=True)

    print(len(val_annotation))
    if not os.path.exists('resized_validation_data'):
        os.makedirs('resized_validation_data')
        
    val_annotations_path = 'resized_validation_data/annotations.json'
    with open(val_annotations_path, 'w') as f:
        json.dump(val_annotation, f, indent=4)
    print(f"Validation images and bounding boxes saved to {val_annotations_path}")

In [10]:
generate_validation()
# image_dir = 'resized_validation_data'
# annotations_path = 'resized_validation_data/annotations.json'
# visualize(image_dir, annotations_path)

80
Validation images and bounding boxes saved to resized_validation_data/annotations.json


# Generate Test Images

In [12]:
def generate_test_image(start_index, image_path, num_images, target_size, resize=False):
    image = cv2.imread(image_path)
    bbox = [0, 0, 0, 0]
    if resize:
        if not os.path.exists('resized_test_data'):
            os.makedirs('resized_test_data')
    else:
        if not os.path.exists('test_data'):
            os.makedirs('test_data')
    for i in range(num_images):
        augmented_image, _ = augment_image(image.copy(), bbox.copy())
        augmented_image = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2RGB)
        if resize:
            augmented_image = cv2.resize(augmented_image, target_size)
        augmented_image_path = f'test_data/image_{i+start_index}.jpg'
        if resize:
            augmented_image_path = f'resized_test_data/image_{i+start_index}.jpg'
        cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

    return image_path

In [16]:
target_size = (256, 256)
generate_test_image(0, 'original_data/back.bmp', 5, target_size, resize=False)
generate_test_image(5, 'original_data/back_1.bmp', 5, target_size, resize=False)
generate_test_image(10, 'original_data/front.bmp', 5, target_size, resize=False)
generate_test_image(15, 'original_data/stain_back.bmp', 5, target_size, resize=False)

'original_data/stain_back.bmp'

In [ ]:
# image_path = 'back.bmp'
# target_size = (256, 256)
# generate_image(image_path, target_size, resize_image=False)
# image_dir = 'data'
# annotations_path = 'data/annotations.json'
# visualize(image_dir, annotations_path)

In [ ]:
# generate_image(image_path, target_size, resize_image=False, validation=True)
# image_dir = 'validation_data'
# annotations_path = 'validation_data/annotations.json'
# visualize(image_dir, annotations_path)


In [ ]:
# generate_image(image_path, target_size, resize_image=True, validation=True)
# image_dir = 'resized_validation_data'
# annotations_path = 'resized_validation_data/annotations.json'
# visualize(image_dir, annotations_path)

In [ ]:
def test_box():
    csv_data = {
        "x": 5.075424261470773,
        "y": 10.69767441860465,
        "width": 19.721648558857858,
        "height": 31.395348837209305,
        "rotation": 0,
        "rectanglelabels": ["CCP"],
        "original_width": 1036,
        "original_height": 646
    }

    # Load the original image
    image_path = 'back.bmp'  # Update with your image path
    image = cv2.imread(image_path)
    image_height, image_width, _ = image.shape

    x_min = int((csv_data["x"] / 100) * image_width)
    y_min = int((csv_data["y"] / 100) * image_height)
    bbox_width = int((csv_data["width"] / 100) * image_width)
    bbox_height = int((csv_data["height"] / 100) * image_height)
    x_max = x_min + bbox_width
    y_max = y_min + bbox_height

    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 125, 0), 2)

    # # Optionally, add label text if available
    label = ','.join(csv_data["rectanglelabels"])
    cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

    # # Display the image with the bounding box
    cv2.imshow('Image with Bounding Box', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    image_path = 'back.bmp'  # Update with your image path
    image = cv2.imread(image_path)
    resized_img = cv2.resize(image, (256, 256))
    cv2.imshow('Resized Image', resized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    original_size = (image_width, image_height)
    target_size = (256, 256)
    bbox = {
        "x_min": x_min,
        "y_min": y_min,
        "x_max": x_max,
        "y_max": y_max
    }
    adjusted_bbox = adjust_bounding_boxes([bbox], original_size, target_size)
    adjusted_bbox = adjusted_bbox[0]
    x_min, y_min, x_max, y_max = [int(x) for x in adjusted_bbox]
    cv2.rectangle(resized_img, (x_min, y_min), (x_max, y_max), (0, 125, 0), 2)

    # # Optionally, add label text if available
    label = ','.join(csv_data["rectanglelabels"])
    cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

    # # Display the image with the bounding box
    cv2.imshow('Resized Image with Bounding Box', resized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # color_jitter = transforms.ColorJitter(
    #     brightness=0.2,    # Randomly change brightness
    #     contrast=0.2,      # Randomly change contrast
    #     saturation=0.2,    # Randomly change saturation
    #     hue=0.1            # Randomly change hue
    # )

    # image = color_jitter(Image.fromarray(image))
    # image = np.array(image)



    '''
    randomAffine = transforms.RandomAffine(
        degrees=10,         # Randomly rotate between -10 and 10 degrees
        translate=(0.1, 0.1),  # Randomly translate by 10% of the image height/width
        scale=(0.8, 1.2),   # Randomly scale image between 0.8 and 1.2 times
        shear=10            # Randomly shear by -10 to 10 degrees
    )


    image = randomAffine(Image.fromarray(image))
    image = np.array(image)
    cv2.imshow('Image with Bounding Box', image)
    '''
    # blur = transforms.GaussianBlur(kernel_size=(5,25), sigma=(0.1, 5))
    # image = blur(Image.fromarray(image))
    # image = np.array(image)
    # image = image.resize((256, 256))
    # cv2.imshow('Image with Bounding Box', image)

    # Convert normalized coordinates to pixel coordinates

    #print(x_min, y_min, x_max, y_max)

    

In [ ]:
# test_box()

In [ ]:
# csv_data = {
#     "x": 5.075424261470773,
#     "y": 10.69767441860465,
#     "width": 19.721648558857858,
#     "height": 31.395348837209305,
#     "rotation": 0,
#     "rectanglelabels": ["CCP"],
#     "original_width": 1036,
#     "original_height": 646
# }

# # Load the original image
# image_path = 'data/back.bmp'  # Update with your image path
# image = cv2.imread(image_path)
# image_height, image_width, _ = image.shape

# # Convert normalized coordinates to pixel coordinates
# x_min = int((csv_data["x"] / 100) * image_width)
# y_min = int((csv_data["y"] / 100) * image_height)
# bbox_width = int((csv_data["width"] / 100) * image_width)
# bbox_height = int((csv_data["height"] / 100) * image_height)
# x_max = x_min + bbox_width
# y_max = y_min + bbox_height

# print(x_min, y_min, x_max, y_max)

# image = np.flip(image, [0, 1])
# image = np.array(image)

# x_min = image_width - x_min
# x_max = image_width - x_max

# y_min = image_height - y_min
# y_max = image_height - y_max

# # Draw bounding box on the image
# cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 125, 0), 2)

# # Optionally, add label text if available
# label = ','.join(csv_data["rectanglelabels"])
# cv2.putText(image, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)

# # Display the image with the bounding box
# cv2.imshow('Image with Bounding Box', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
# csv_data = {
#     "x": 5.075424261470773,
#     "y": 10.69767441860465,
#     "width": 19.721648558857858,
#     "height": 31.395348837209305,
#     "rotation": 0,
#     "rectanglelabels": ["CCP"],
#     "original_width": 1036,
#     "original_height": 646
# }

# image_path = 'back.bmp'  # Update with the actual path
# image = cv2.imread(image_path)
# # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for visualization

# bbox = normalized_to_pixel_bbox(csv_data, csv_data["original_width"], csv_data["original_height"])

# num_train_augmentations = 50  # Number of augmentations to generate
# train_augmented_results = []

# for i in range(num_train_augmentations):
#     augmented_image, adjusted_bbox = augment_image(image.copy(), bbox.copy())
#     augmented_image_np = np.array(augmented_image)
#     train_augmented_results.append((augmented_image_np, adjusted_bbox))
#     # print(adjusted_bbox)
#     # Display the augmented image with the bounding box
#     # augmented_image_path = f'augmented_image_{i}.jpg'
#     # augmented_image_bgr = cv2.cvtColor(augmented_image_np, cv2.COLOR_RGB2BGR)
#     # cv2.rectangle(augmented_image_bgr, (int(adjusted_bbox[0]), int(adjusted_bbox[1])), 
#     #           (int(adjusted_bbox[2]), int(adjusted_bbox[3])), (255, 0, 0), 2)
#     # cv2.imshow('Augmented Image', augmented_image_bgr)
#     # cv2.waitKey(0)
#     # cv2.destroyAllWindows()
#     # cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

# train_bbox_annotations = []
# for i, (augmented_image, adjusted_bbox) in enumerate(train_augmented_results):
#     # Save augmented image
#     augmented_image_path = f'data/augmented_image_{i}.jpg'
#     image_name = f'augmented_image_{i}.jpg'
#     cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

    
#     # Save adjusted bounding box coordinates to the annotations list
#     bbox_data = {
#         'image_path': image_name,
#         'bbox': {
#             'x_min': adjusted_bbox[0],
#             'y_min': adjusted_bbox[1],
#             'x_max': adjusted_bbox[2],
#             'y_max': adjusted_bbox[3]
#         }
#     }
#     train_bbox_annotations.append(bbox_data)

# # Save bounding box annotations to a JSON file
# train_annotations_path = 'data/annotations.json'
# with open(train_annotations_path, 'w') as f:
#     json.dump(train_bbox_annotations, f, indent=4)

# print(f"Training images and bounding boxes saved to 'data/' directory.")


# num_val_augmentations = 10  # Number of augmentations to generate
# val_augmented_results = []

# for i in range(num_val_augmentations):
#     augmented_image, adjusted_bbox = augment_image(image.copy(), bbox.copy())
#     augmented_image_np = np.array(augmented_image)
#     val_augmented_results.append((augmented_image_np, adjusted_bbox))
#     # print(adjusted_bbox)
#     # Display the augmented image with the bounding box
#     # augmented_image_path = f'augmented_image_{i}.jpg'
#     # augmented_image_bgr = cv2.cvtColor(augmented_image_np, cv2.COLOR_RGB2BGR)
#     # cv2.rectangle(augmented_image_bgr, (int(adjusted_bbox[0]), int(adjusted_bbox[1])), 
#     #           (int(adjusted_bbox[2]), int(adjusted_bbox[3])), (255, 0, 0), 2)
#     # cv2.imshow('Augmented Image', augmented_image_bgr)
#     # cv2.waitKey(0)
#     # cv2.destroyAllWindows()
#     # cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

# val_bbox_annotations = []
# for i, (augmented_image, adjusted_bbox) in enumerate(val_augmented_results):
#     # Save augmented image
#     augmented_image_path = f'val_data/augmented_image_{i}.jpg'
#     image_name = f'augmented_image_{i}.jpg'
#     cv2.imwrite(augmented_image_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))

    
#     # Save adjusted bounding box coordinates to the annotations list
#     bbox_data = {
#         'image_path': image_name,
#         'bbox': {
#             'x_min': adjusted_bbox[0],
#             'y_min': adjusted_bbox[1],
#             'x_max': adjusted_bbox[2],
#             'y_max': adjusted_bbox[3]
#         }
#     }
#     val_bbox_annotations.append(bbox_data)

# # Save bounding box annotations to a JSON file
# val_annotations_path = 'val_data/val_annotations.json'
# with open(val_annotations_path, 'w') as f:
#     json.dump(val_bbox_annotations, f, indent=4)


# print(f"Validation images and bounding boxes saved to 'val_data/' directory.")

